# Eindopdracht Vision
In dit project onderzoek ik de impact van verschillende activatiefuncties op de nauwkeurigheid van een convolutioneel neuraal netwerk (CNN) voor beeldclassificatie en bounding box voorspellingen met TensorFlow. Activatiefuncties zijn cruciaal omdat ze niet-lineaire transformaties toepassen op neuronuitvoer, waardoor complexe patronen in data kunnen worden gemodelleerd.

Het doel is om te bepalen welke activatiefunctie(s) het beste presteren voor mijn specifieke taken van beeldclassificatie en het voorspellen van bounding boxes. Ik zal veelgebruikte functies zoals ReLU, Sigmoid, Softmax en Swish vergelijken om te zien welke de hoogste nauwkeurigheid bereiken bij het classificeren van afbeeldingen en het voorspellen van bounding boxes. Daarnaast onderzoek ik hoe variaties in inputafbeeldingen (zoals zwart-wit conversie, rotatie en andere augmentaties) de modelprestaties beïnvloeden.

Deze experimenten zullen niet alleen de prestaties van mijn CNN optimaliseren, maar ook inzicht verschaffen in hoe activatiefuncties en variaties in input gezamenlijk de algehele nauwkeurigheid van het model voor beeldclassificatie en bounding box voorspellingen beïnvloeden.

## Background
Activatiefuncties zijn essentieel in neurale netwerken omdat ze bepalen of een neuron moet worden geactiveerd op basis van zijn input naar het netwerk. Deze functies gebruiken wiskundige bewerkingen om te beslissen of de input belangrijk is voor de voorspelling. Niet-lineaire activatiefuncties stellen neurale netwerken in staat complexe patronen te leren die vaak voorkomen in echte gegevens, in tegenstelling tot lineaire functies die beperkt zijn in hun vermogen om dergelijke patronen te vangen.

Data augmentatie is een krachtig middel tegen overfitting in machine learning-modellen, met name in computer vision. Door nieuwe trainingsvoorbeelden te genereren via transformaties van bestaande data, vergroot data augmentatie effectief de omvang en diversiteit van de trainingsdataset. Hierdoor kunnen modellen beter generaliseren naar nieuwe, ongeziene data zonder terug te hoeven keren naar kostbare datacollectie. Dit maakt data augmentatie een kosteneffectieve oplossing voor het verbeteren van de prestaties en robuustheid van machine learning-modellen in diverse toepassingsgebieden zoals beeldherkenning en automatisering.

## methodebeschrijving


In [ ]:
import matplotlib.pyplot as plt
from matplotlib import patches
import numpy as np
from PIL import Image
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint 
import pandas as pd
from pathlib import Path
from IPython.display import display, clear_output
import os

NUM_CLASSES = 3
TRAINPATH = 'dataset/train/'
VALIDPATH = 'dataset/valid/'
TESTPATH = 'dataset/test/'
NEWPATH = 'dataset/new/'

def CraeteLabelMap(path):
    annotations = pd.read_csv(path + 'annotation.csv')
    # Map label strings to integer labels
    labelMap = {}
    unique_labels = set(annotations['class'])
    for i, label in enumerate(unique_labels):
        labelMap[label] = i
    return labelMap
    
def readData(path, labelMap):
    target_size = (224, 224)
    annotations = pd.read_csv(path + 'annotation.csv')

    X_img = []
    y_labels = []
    y_bboxes = []

    for i, annotation in annotations.iterrows():
        image_path = annotation['filename']
        labels = annotation['class']
        xmin = annotation['xmin']
        ymin = annotation['ymin']
        xmax = annotation['xmax']
        ymax = annotation['ymax']

        # Preprocess the image
        image = Image.open(path + image_path)
        image = image.resize(target_size)
        image_data = np.array(image, dtype=np.float32)
        image_data /= 255.0

        # Normalize bounding box coordinates
        height, width, channels = image_data.shape
        xmin_norm = xmin / width
        ymin_norm = ymin / height
        xmax_norm = xmax / width
        ymax_norm = ymax / height
        
        X_img.append(image_data)
        y_labels.append(labelMap[labels])
        y_bboxes.append([xmin_norm, ymin_norm, xmax_norm, ymax_norm])

    X_img = np.array(X_img)
    y_labels = np.array(y_labels)
    y_bboxes = np.array(y_bboxes)
    
    # Convert labels to one-hot encoding
    Labels_cls = to_categorical(y_labels, NUM_CLASSES)

    return X_img, Labels_cls, y_bboxes

labelMap = CraeteLabelMap(TRAINPATH)

trainImg, trainLabels, trainbboxes = readData(TRAINPATH, labelMap)
validImg, validLabels, validbboxes = readData(VALIDPATH, labelMap)
testImg, testLabels, testbboxes = readData(TESTPATH, labelMap)

* CraeteLabelMap(path): Deze functie leest de annotatie CSV-bestand en maakt een map van unieke labels naar integerwaarden.
* readData(path, labelMap): Deze functie leest de afbeeldingen en annotaties, schaalt de afbeeldingen naar 224x224 pixels, normaliseert de bounding box coördinaten en converteert de labels naar one-hot encoding.

In [ ]:
def augmentImages(images, labels, bboxes):
    augmentedImages = []
    augmentedLabels = []
    augmentedBboxes = []

    for img, label, bbox in zip(images, labels, bboxes):
        # Original image
        augmentedImages.append(img)
        augmentedLabels.append(label)
        augmentedBboxes.append(bbox)
        
        # Grayscale image
        gray_img = np.mean(img, axis=2, keepdims=True)
        gray_img = np.repeat(gray_img, 3, axis=2)
        augmentedImages.append(gray_img)
        augmentedLabels.append(label)
        augmentedBboxes.append(bbox)

        # # Inverted image
        # inverted_img = 1.0 - img
        # augmentedImages.append(inverted_img)
        # augmentedLabels.append(label)
        # augmentedBboxes.append(bbox)
        
        # Rotate the image by 90 degrees
        rotated_img = np.array(Image.fromarray((img * 255).astype(np.uint8)).rotate(90)) / 255.0
        augmentedImages.append(rotated_img)
        augmentedLabels.append(label)
        xmin, ymin, xmax, ymax = bbox
        # Swap and adjust coordinates for 90-degree rotation
        new_xmin = ymin  # New xmin = old ymin
        new_ymin = 1 - xmax  # New ymin = 1 - old xmax
        new_xmax = ymax  # New xmax = old ymax
        new_ymax = 1 - xmin  # New ymax = 1 - old xmin
        augmentedBboxes.append([new_xmin, new_ymin, new_xmax, new_ymax])
        

    return np.array(augmentedImages), np.array(augmentedLabels), np.array(augmentedBboxes)

augmentedTrainImg, augmentedTrainLabels, augmentedTrainbboxes = augmentImages(trainImg, trainLabels, trainbboxes)
augmentedValidImg, augmentedValidLabels, augmentedValidbboxes = augmentImages(validImg, validLabels, validbboxes)
augmentedTestImg, augmentedTestLabels, augmentedTestbboxes = augmentImages(testImg, testLabels, testbboxes)

* augment_images(images, labels, bboxes): Deze functie voegt nieuwe varianten van de afbeeldingen toe door grijswaardenconversie en rotatie toe te passen, en past de bijbehorende bounding box coördinaten aan.

In [ ]:
def getModel():
    preModel = Path("models/softmax.h5")
    if preModel.is_file():
        # load trained model
        model = load_model('models/softmax.h5')
        return model
    else:
        # Define input tensor
        inputs = Input(shape=(224, 224, 3))
        
        # Convolutional layers
        x = Conv2D(32, kernel_size=(3, 3), activation="relu")(inputs)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Conv2D(64, kernel_size=(3, 3), activation="relu")(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Conv2D(128, kernel_size=(3, 3), activation="relu")(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        
        # Define the classification prediction head
        classification = Flatten()(x)
        classification = Dense(256, activation="relu")(classification)
        classification = Dropout(0.5)(classification)
        class_output = Dense(NUM_CLASSES, activation="softmax", name="class_output")(classification)
        
        # Define the bounding box prediction head
        bbox = Flatten()(x)
        bbox = Dense(256, activation="relu")(bbox)
        bbox = Dropout(0.5)(bbox)
        bbox_output = Dense(4, activation="linear", name="bbox_output")(bbox)
        
        
        # Define the model with input and output layers
        model = Model(inputs=inputs, outputs=(class_output, bbox_output))
        
        losses = {
            'class_output': 'categorical_crossentropy',
        	'bbox_output': 'mean_squared_error'
        }
        
        metrics={
            'class_output': 'accuracy',
            'bbox_output': 'accuracy'
        }
        
        trainTargets = {
        	"class_output": trainLabels,
        	"bbox_output": trainbboxes
            # "class_output": augmentedTrainLabels,
        	# "bbox_output": augmentedTrainbboxes
        }
        validTargets = {
        	"class_output": validLabels,
        	"bbox_output": validbboxes
            # "class_output": augmentedValidLabels,
        	# "bbox_output": augmentedValidbboxes
        }
        
        # Compile the model with categorical_crossentropy, mean_squared_error and Adam optimizer
        model.compile(loss=losses, optimizer='adam', metrics=metrics)
        
        # Train the model on the training data and validation on validation data
        checkpoint = ModelCheckpoint('models/softmax.h5', monitor='bbox_output_accuracy', save_best_only=True, mode='max', verbose=1)
        model.fit(trainImg, trainTargets, epochs=150, batch_size=50, validation_data=(validImg, validTargets), callbacks=[checkpoint])
        # model.fit(augmentedTrainImg, trainTargets, epochs=150, batch_size=16, validation_data=(augmentedValidImg, validTargets), callbacks=[checkpoint])
        
        return model
    
def testModel(model):
    testTargets = {
        "class_output": testLabels,
        "bbox_output": testbboxes
        # "class_output": augmentedTestLabels,
        # "bbox_output": augmentedTestbboxes
    }

    # Evaluate the model on the test set
    test_loss, test_bbox_loss, test_class_loss, test_bbox_acc, test_class_acc = model.evaluate(testImg, testTargets, verbose=0)

    # Print the test set metrics
    print('Test set loss: ', test_loss)
    print('Test set bounding box loss: ', test_bbox_loss)
    print('Test set class loss: ', test_class_loss)
    print('Test set bounding box accuracy: ', test_bbox_acc)
    print('Test set class accuracy: ', test_class_acc)


model = getModel()
testModel(model)


* getModel(): Deze functie definieert de architectuur van het CNN, bestaande uit convolutionele en max-pooling lagen, gevolgd door aparte dense lagen voor classificatie en bounding box voorspelling. Het model wordt gecompileerd met 'categorical_crossentropy' voor classificatie en 'mean_squared_error' voor bounding box voorspelling, en getraind op de trainingsdata.
* testModel(model): Deze functie evalueert het getrainde model op de testset en print de verlieswaarden en nauwkeurigheden voor zowel classificatie als bounding box voorspelling.

In [ ]:
def unnormalize_bbox(bbox, origSize, resSize):
    scale_x = origSize[0] / resSize
    scale_y = origSize[1] / resSize

    xNorm, yNorm, wNorm, hNorm = bbox
    xmin = xNorm * (origSize[0] / scale_x)
    ymin = yNorm * (origSize[1] / scale_y)
    xmax = wNorm * (origSize[0] / scale_x)
    ymax = hNorm * (origSize[1] / scale_y)

    return (xmin, ymin, xmax, ymax)

def scaleBbox(bbox, origSize, resSize):
    # Calculate scaling factor
    scaleX = resSize[0] / origSize[0]
    scaleY = resSize[1] / origSize[1]

    xmin, ymin, xmax, ymax = bbox
    xminScaled = xmin * scaleX
    yminScaled = ymin * scaleY
    xmaxScaled = xmax * scaleX
    ymaxScaled = ymax * scaleY

    return (xminScaled, yminScaled, xmaxScaled, ymaxScaled)

def readNewImages(image_dir):
    target_size = (224, 224)
    X_img = []

    # List all image files
    image_files = [file for file in os.listdir(image_dir) if file.endswith('.jpg')]
    
    # Shuffle the list randomly
    np.random.shuffle(image_files)

    for image_file in image_files:
        image = Image.open(os.path.join(image_dir, image_file))
        image = image.resize(target_size)
        image_data = np.array(image, dtype=np.float32)
        image_data /= 255.0

        X_img.append(image_data)

    X_img = np.array(X_img)
    return X_img

# Load the new data
newImg = readNewImages(NEWPATH)

* unnormalize_bbox(bbox, origSize, resSize): Deze functie schaalt genormaliseerde bounding box-coördinaten terug naar originele afmetingen.
* scaleBbox(bbox, origSize, resSize): Deze functie schaalt bounding box-coördinaten van originele afmetingen naar een ander resolutieniveau.
* readNewImages(image_dir): Deze functie leest nieuwe afbeeldingen in vanuit een opgegeven map en past dezelfde voorverwerking toe als bij de trainings- en testdatasets.

In [ ]:
def prediction(img, labelMap):
    # Predict bounding boxes on the new data
    pred_labels, pred_bboxes = model.predict(img)

    orig_size = (1920, 1200)
    res_size = 224
    bboxes = [unnormalize_bbox(bbox, orig_size, res_size) for bbox in pred_bboxes]
    img_size = (224, 224)
    bboxScaled = [scaleBbox(bbox, orig_size, img_size) for bbox in bboxes]

    labelMap = {v: k for k, v in labelMap.items()}

    # Generate plot objects for each image and display them one by one
    for i in range(len(img)):
        fig, ax = plt.subplots()

        # Ensure the image has the correct shape
        current_img = img[i]
        
        if current_img.shape == (3,):  # If it has shape (3,), assume grayscale and reshape
            current_img = current_img.reshape(current_img.shape + (1,))

        lbl = np.argmax(pred_labels[i], axis=-1)
        ax.imshow(current_img)  # Use the reshaped image
        ax.axis('off')
        label_name = labelMap[lbl] 
        xmin, ymin, xmax, ymax = bboxScaled[i]
        ax.text(xmin, ymin, label_name, fontsize=10, color='red')
        rect = patches.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, fill=False, edgecolor='r')
        ax.add_patch(rect)

        # Display the plot in the notebook
        display(fig)
        plt.close(fig)

        # Wait for user input (press Enter) to continue to the next image
        input("Press Enter to continue to the next image...")
        clear_output(wait=True)

# Call the function
prediction(newImg, labelMap)

* prediction(img, labelMap): Deze functie maakt voorspellingen met behulp van het opgegeven model op nieuwe afbeeldingen en visualiseert de resultaten met behulp van bounding boxes en voorspelde klasselabels.

## Experiment 1: Evaluatie van Activatiefuncties voor Classificatie
Dit experiment evalueert de invloed van verschillende activatiefuncties (softmax, sigmoid en swish) op de boundingbox en classificatieprestaties van het CNN-model.

### softmax
De softmax-functie transformeert een vector van getallen naar waarden tussen 0 en 1, waarbij de som van deze waarden gelijk is aan 1. Hierdoor kan de uitvoer worden geïnterpreteerd als de kans dat een specifieke invoer tot een bepaalde klasse behoort.

De softmax-functie kan worden gedefinieerd als:
$$
\text{softmax}(x_i) = \frac{e^{x_i}}{\sum_{j=1}^{n} e^{x_j}}
$$

Waarbij \( x_i \) de \( i \)-waarde in de inputvector is en \( n \) de dimensie van de vector is.

De Softmax-activatiefunctie wordt gebruikt in de output laag voor classificatie. De Softmax-activatiefunctie berekent de relatieve waarschijnlijkheden voor welke klasse het zou zijn.



In [ ]:
# Define input tensor
inputs = Input(shape=(224, 224, 3))

# Convolutional layers
x = Conv2D(32, kernel_size=(3, 3), activation="relu")(inputs)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(64, kernel_size=(3, 3), activation="relu")(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(128, kernel_size=(3, 3), activation="relu")(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

# Define the classification prediction head
classification = Flatten()(x)
classification = Dense(256, activation="relu")(classification)
classification = Dropout(0.5)(classification)
class_output = Dense(NUM_CLASSES, activation="softmax", name="class_output")(classification)

# Define the bounding box prediction head
bbox = Flatten()(x)
bbox = Dense(256, activation="relu")(bbox)
bbox = Dropout(0.5)(bbox)
bbox_output = Dense(4, activation="linear", name="bbox_output")(bbox)

<img src="experiment_Image/softmax_predict.png" alt="softmax_predict" width="500" heigh="500"/>
<img src="experiment_Image/softmax_evaluate1_train.png" alt="softmax_train" width="500" heigh="500"/>


### sigmoid
Sigmoid zet de inputwaarde om naar een waarde tussen 0 en 1. Deze omzetting maakt het mogelijk om de uitvoer te interpreteren als de kans dat een gegeven invoer tot een bepaalde klasse behoort. Als de uitvoerwaarde dichter bij 0 ligt, wordt de invoer toegewezen aan de ene klasse, terwijl een waarde dichter bij 1 de toewijzing aan een andere klasse aanduidt. Dit maakt de sigmoid-functie ideaal voor binaire classificatie.

De wiskundige formule van de sigmoid-functie is:
$$
\sigma(x) = \frac{1}{1 + e^{-x}}
$$

Hier is \( x \)  de inputwaarde.



In [ ]:
# Define input tensor
inputs = Input(shape=(224, 224, 3))

# # Convolutional layers
x = Conv2D(32, kernel_size=(3, 3), activation="relu")(inputs)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(64, kernel_size=(3, 3), activation="relu")(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(128, kernel_size=(3, 3), activation="relu")(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

# Define the classification and bounding box prediction heads
classification = Flatten()(x)
classification = Dense(256, activation="relu")(classification)
classification = Dropout(0.5)(classification)
class_output = Dense(NUM_CLASSES, activation="sigmoid", name="class_output")(classification)

bbox = Flatten()(x)
bbox = Dense(256, activation="relu")(bbox)
bbox = Dropout(0.5)(bbox)
bbox_output = Dense(4, activation="linear", name="bbox_output")(bbox)

<img src="experiment_Image/sigmoid_predict2.png" alt="sigmoid_predict2" width="500" heigh="500"/>
<img src="experiment_Image/sigmoid_evaluate1_train.png" alt="sigmoid_train" width="500" heigh="500"/>

### swish

De swish-functie kan worden gedefinieerd als:

$$f(x) = x \cdot \sigma(x)$$
$$\text{where} \quad \sigma(x) = \frac{1}{1+e^{-x}}$$



In [ ]:
# Define input tensor
inputs = Input(shape=(224, 224, 3))

# # Convolutional layers
x = Conv2D(32, kernel_size=(3, 3), activation="relu")(inputs)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(64, kernel_size=(3, 3), activation="relu")(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(128, kernel_size=(3, 3), activation="relu")(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

# Define the classification and bounding box prediction heads
classification = Flatten()(x)
classification = Dense(256, activation="swish")(classification)
classification = Dropout(0.5)(classification)
class_output = Dense(NUM_CLASSES, activation="sigmoid", name="class_output")(classification)

bbox = Flatten()(x)
bbox = Dense(256, activation="swish")(bbox)
bbox = Dropout(0.5)(bbox)
bbox_output = Dense(4, activation="linear", name="bbox_output")(bbox)


<img src="experiment_Image/swish_predict.png" alt="swish_predict" width="500" heigh="500"/>
<img src="experiment_iamge_fix/swish_softmax_evaluate1_train.png" alt="swish_softmax_evaluate1_train" width="500" heigh="500"/>

<img src="experiment_Image/Class_accuracy.png" alt="Class_accuracy" width="500" heigh="500"/>
<img src="experiment_Image/Bbox_accuracy.png" alt="Bbox_accuracy" width="500" heigh="500"/>

Uit de resultaten blijkt dat Softmax en Sigmoid vergelijkbare prestaties leveren voor classificatie, terwijl Swish aanzienlijk minder goed presteert voor zowel classificatie als het voorspellen van bounding boxes. Dit suggereert dat Softmax de voorkeur verdient voor classificatiedoeleinden, vooral in combinatie met de ReLU-activatiefunctie.

## Experiment 2: Augments
In dit experiment onderzoeken we de impact van verschillende beeldaugmentaties op de nauwkeurigheid van het model. We zullen het model trainen met drie verschillende vormen van augmentaties: grayscale, rotatie en inverted images.

<img src="experiment2_Image/grayscale.png" alt="grayscale" width="500" heigh="500"/>
<img src="experiment2_Image/inverted.png" alt="inverted" width="500" heigh="500"/>
<img src="experiment2_Image/grayscale_invert_evaluate.png" alt="grayscale_invert_evaluate" width="500" heigh="500"/>

Hier hebben we het model getraind met zowel de originele afbeeldingen als met grayscale en inverted versies ervan.

<img src="experiment2_Image/grayscale.png" alt="grayscale" width="500" heigh="500"/>
<img src="experiment2_Image/rotate.png" alt="rotate" width="500" heigh="500"/>
<img src="experiment2_Image/grayscale_rotate_evaluate.png" alt="grayscale_rotate_evaluate" width="500" heigh="500"/>

<img src="experiment2_Image/Class_accuracy.png" alt="Class_accuracy" width="500" heigh="500"/>
<img src="experiment2_Image/bbox_accuracy.png" alt="bbox_accuracy" width="500" heigh="500"/>

Uit de resultaten blijkt dat de toevoeging van augmentaties een variërende invloed heeft op de nauwkeurigheid van het model. Specifiek zien we dat de combinatie van grayscale en rotatie resulteert in een verbetering van de classificatienauwkeurigheid. Echter, wanneer grayscale wordt gecombineerd met inverted images, vertoont de classificatienauwkeurigheid een lichte afname in vergelijking met het model zonder augmentaties. Wat betreft de bounding box-nauwkeurigheid laten beide combinaties een afname zien ten opzichte van het basismodel zonder augmentaties.

## literatuurlijst
1. Adrian Rosebrock *Object detection: Bounding box regression with Keras, TensorFlow, and Deep Learning* 2020.
   - URL: [https://pyimagesearch.com/2020/10/05/object-detection-bounding-box-regression-with-keras-tensorflow-and-deep-learning/](https://pyimagesearch.com/2020/10/05/object-detection-bounding-box-regression-with-keras-tensorflow-and-deep-learning/)

2. Adrian Rosebrock *Multi-class object detection and bounding box regression with Keras, TensorFlow, and Deep Learning* 2020.
   - URL: [https://pyimagesearch.com/2020/10/12/multi-class-object-detection-and-bounding-box-regression-with-keras-tensorflow-and-deep-learning/](https://pyimagesearch.com/2020/10/12/multi-class-object-detection-and-bounding-box-regression-with-keras-tensorflow-and-deep-learning/)

3. Rohit Thakur *Step by step VGG16 implementation in Keras for beginners* 2019.
   - URL: [https://towardsdatascience.com/step-by-step-vgg16-implementation-in-keras-for-beginners-a833c686ae6c](https://towardsdatascience.com/step-by-step-vgg16-implementation-in-keras-for-beginners-a833c686ae6c)

4. Rohit Thakur *Beginner’s Guide to VGG16 Implementation in Keras* 2023.
   - URL: [https://builtin.com/machine-learning/vgg16](https://builtin.com/machine-learning/vgg16)

5. Shipra Saxena *Introduction to Softmax for Neural Network* 2023.
   - URL: [https://www.analyticsvidhya.com/blog/2021/04/introduction-to-softmax-for-neural-network/](https://www.analyticsvidhya.com/blog/2021/04/introduction-to-softmax-for-neural-network/)

6. Pragati Baheti *Activation Functions in Neural Networks [12 Types & Use Cases]* 2021
 - URL: [https://www.v7labs.com/blog/neural-networks-activation-functions](https://www.v7labs.com/blog/neural-networks-activation-functions)

## bijlage